In [51]:
from wrangle import *


from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression 
from math import sqrt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

df=prep_zillow_2017(1.25)

x_train_and_validate, x_test = train_test_split(df, random_state=123)
x_train, x_validate = train_test_split(x_train_and_validate)
df=x_train



Number of observations removed: 471854
This is our percent change after removing all the outliers and then the nulls:
 -22.02%
mean kurt:
-0.2173658225527361


None

In [52]:



cols=set(df.columns.to_list())

nums=cols-{
'fips',                          
'bedroomcnt',                   
'bathroomcnt',
'decade',
'yearbuilt'}

cats=cols-nums

cat_2_combs=list(combinations(cats,2))
num_2_combs=list(combinations(nums,2))
cat_3_combs=list(combinations(cats,3))
num_3_combs=list(combinations(nums,3))
cat_v_num=set(product(cats,nums))

num_v_cat=set(product(nums,cats))



cols=list(cols)
cats=list(cats)
nums=list(nums)



In [53]:
print(f'nums:\n{nums}\n\ncats:\n{cats}\n\n')

nums:
['taxamount', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt']

cats:
['yearbuilt', 'decade', 'bathroomcnt', 'fips', 'bedroomcnt']





### Exercises

Do you work for this exercise in either a jupyter notebook named evaluate within your regression-exercises repository. By the end of this exercise you will also create a python script named evaluate, so the overall deliverables for this exercise are the python script and jupyter notebook both with the name evaluate within your regression-exercises repo.



1. Load your zillow dataset.
2. Fit a linear regression model (ordinary least squares) and compute $\hat{y}$, predictions of taxvaluedollarcnt using only calculatedfinishedsqft.  \
Here is some sample code to get you started:

> ```py
> from sklearn.linear_model import LinearRegression
> 
> # assuming X and y are already defined
> 
> model = LinearRegression().fit(X, y)
> 
> predictions = model.predict(X)
> ```


 
 Modify and add to the code above as necessary for it to work with the zillow dataset.

3. Plot the residuals for the linear regression model that you made.

4. Calculate the sum of squared errors, explained sum of squares, total sum of squares, mean squared error, and root mean squared error for your model.

5. Calculate the sum of squared errors, mean squared error, and root mean squared error for the baseline model (i.e. a model that always predicts the average taxvaluedollarcnt amount).

6. Write python code that compares the sum of squared errors for your model against the sum of squared errors for the baseline model and outputs whether or not your model performs better than the baseline model.

7. What is the amount of variance explained in your model?

8. Is your model better than the baseline model?

9. Create a file named evaluate.py that contains the following functions.
    * plot_residuals(y, $\hat{y}$): creates a residual plot
    * regression_errors(y, $\hat{y}$): returns the following values:
        * sum of squared errors (SSE)
        * explained sum of squares (ESS)
        * total sum of squares (TSS)
        * mean squared error (MSE)
        * root mean squared error (RMSE)
    * baseline_mean_errors(y): computes the SSE, MSE, and RMSE for the baseline model
    * better_than_baseline(y, $\hat{y}$): returns true if your model performs better than the baseline, otherwise false


#### Bonus Exercises



1. Load the tips dataset from either pydataset or seaborn. Fit a linear regression model (ordinary least squares) and compute $\hat{y}$, predictions of tip using total_bill. 
2. Load the mpg dataset and fit a model that predicts highway mileage based on engine displacement. Take a look at all the regression evaluation metrics, and determine whether this model is better than the baseline model. Use the functions from your evaluate.py to help accomplish this.

In [54]:
dfa=df
dfa.drop(columns=cats,inplace=True)
dfa.drop(columns='taxamount',inplace=True)

dfa.rename(columns={'calculatedfinishedsquarefeet':'x',	'taxvaluedollarcnt':'y'},inplace=True)
df['yhat_baseline'] = df['y'].mean()
dfa.head()

,x,y,yhat_baseline
1487485,2.14e+03,1.42e+05,3.31e+05
328322,1.22e+03,3.34e+05,3.31e+05
368981,1.63e+03,6.57e+05,3.31e+05
93871,1.01e+03,5.76e+05,3.31e+05
433187,1.45e+03,2.78e+05,3.31e+05


In [55]:
# assuming X and y are already defined

ols_model = LinearRegression().fit(dfa[['x']], dfa.y)

In [56]:

dfa['yhat']=ols_model.predict(df[['x']])
dfa.head()

,x,y,yhat_baseline,yhat
1487485,2.14e+03,1.42e+05,3.31e+05,4.06e+05
328322,1.22e+03,3.34e+05,3.31e+05,2.65e+05
368981,1.63e+03,6.57e+05,3.31e+05,3.28e+05
93871,1.01e+03,5.76e+05,3.31e+05,2.34e+05
433187,1.45e+03,2.78e+05,3.31e+05,3.01e+05


In [57]:
MSE2_baseline = mean_squared_error(dfa.y, dfa['yhat_baseline'])
MSE2 = mean_squared_error(dfa.y, dfa.yhat)
SSE2_baseline =  MSE2_baseline * len(dfa)
SSE2 = MSE2 * len(df)
RMSE2_baseline = MSE2_baseline**.5
RMSE2 = MSE2**.5

In [58]:


display(
mean_squared_error(dfa.y, dfa['yhat_baseline'], squared=False),
mean_squared_error(dfa.y, dfa.yhat, squared=False)
)

205400.8680548989

186480.18607925632

In [59]:
MSEetclistnames=["MSE baseline:","MSE:","SSE baseline:" ,
"SSE:", 
"RMSE baseline:", 
"RMSE:"]
MSEetclist=[MSE2_baseline,MSE2,SSE2_baseline,SSE2,RMSE2_baseline,RMSE2]
MSEetclist=[f'{i:.3g}' for i in MSEetclist]
MSEetclistnames=[i+'\n' for i in MSEetclistnames]
MSEtodisp=([MSEetclistnames[i]+MSEetclist[i] for i,k in enumerate(MSEetclistnames)])
[print(f'{i}\n') for i in MSEtodisp]


MSE baseline:
4.22e+10

MSE:
3.48e+10

SSE baseline:
3.98e+16

SSE:
3.28e+16

RMSE baseline:
2.05e+05

RMSE:
1.86e+05



[None, None, None, None, None, None]

In [91]:

dfa['residual'] = dfa['yhat'] - dfa['y']
dfa['residual_baseline'] = dfa['yhat_baseline'] - dfa['y']
dfa['residual^2'] = dfa.residual ** 2
dfa['residual_baseline^2'] = dfa.residual_baseline ** 2
dfa['SSE'] = sum(dfa['residual^2'])
dfa['SSE_baseline'] = sum(df['residual_baseline^2'])
dfa['MSE'] = dfa['SSE'].divide(other = len(dfa))
dfa['MSE_baseline'] = dfa['SSE_baseline'].divide(other = len(dfa))
dfa['RMSE'] = dfa['MSE'].apply(sqrt)
dfa['RMSE_baseline'] = dfa['MSE_baseline'].apply(sqrt)
# dfa['ESS']=sum()
pd.options.display.float_format = '{:,.4g}'.format

dfa.head()



,x,y,yhat_baseline,yhat,residual,residual_baseline,residual^2,residual_baseline^2,SSE,SSE_baseline,MSE,MSE_baseline,RMSE,RMSE_baseline
1487485,"2,139",1.424e+05,3.315e+05,4.062e+05,2.637e+05,1.89e+05,6.955e+10,3.572e+10,3.284e+16,3.984e+16,3.477e+10,4.219e+10,1.865e+05,2.054e+05
328322,"1,215",3.344e+05,3.315e+05,2.652e+05,-6.923e+04,"-2,982",4.793e+09,8.89e+06,3.284e+16,3.984e+16,3.477e+10,4.219e+10,1.865e+05,2.054e+05
368981,"1,628",6.568e+05,3.315e+05,3.282e+05,-3.286e+05,-3.254e+05,1.08e+11,1.059e+11,3.284e+16,3.984e+16,3.477e+10,4.219e+10,1.865e+05,2.054e+05
93871,"1,009",5.756e+05,3.315e+05,2.338e+05,-3.419e+05,-2.442e+05,1.169e+11,5.963e+10,3.284e+16,3.984e+16,3.477e+10,4.219e+10,1.865e+05,2.054e+05
433187,"1,450",2.775e+05,3.315e+05,3.011e+05,2.354e+04,5.394e+04,5.543e+08,2.91e+09,3.284e+16,3.984e+16,3.477e+10,4.219e+10,1.865e+05,2.054e+05


In [111]:

dfError=dfa[['SSE','MSE','RMSE']].head(1)
dfError=dfError.T
dfError.rename_axis('metric',inplace=True)
dfError.rename(columns={dfError.columns[0]:'model_error'},inplace=True)
dfError['baseline'] = dfa[['SSE_baseline', 'MSE_baseline', 'RMSE_baseline']].head(1).values.T
dfError['error_delta']=dfError['model_error']-dfError['baseline']
dfError.sort_values(by='error_delta', ascending=False)

,model_error,baseline,error_delta
metric,,,
RMSE,1.865e+05,2.054e+05,-1.892e+04
MSE,3.477e+10,4.219e+10,-7.415e+09
SSE,3.284e+16,3.984e+16,-7.002e+15


In [110]:
r2_score(dfa.y, dfa.yhat)


0.1757464269679514

In [ ]:


# sklearn.metrics.explained_variance_score

evs = explained_variance_score(dfa.y, dfa.yhat)
print('Explained Variance = ', round(evs,3))